In [34]:
pip install selenium==4.8.2

Note: you may need to restart the kernel to use updated packages.


In [8]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [35]:
#크롬 브라우저 옵션 설정 위해 인스턴스 생성
chrome_options = Options()

#chrome 브라우저 webdriver 세션 종료하더라도 브라우저를 계속 실행하도록
chrome_options.add_experimental_option("detach", True)
#로깅을 비활성화
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

#chrome_options 매개변수를통해 전달되어 설정된 옵션을 사용하도록
driver = webdriver.Chrome(options=chrome_options)

In [11]:
# 무신사 페이지 열기
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.maximize_window() # 브라우저의 크기 전체화면으로 확대

In [ ]:
codimaps = []
style_list_items = driver.find_elements(By.CLASS_NAME, "style-list-item")
    
for style_list_item in style_list_items:
        codimap = {}

        #class name이 style_list_information 인 div에서 codimap_title과 codimap_category 정보 가져오기
        style_list_information = style_list_item.find_element(By.CLASS_NAME, "style-list-information")
        codimap['codimap_category'] = style_list_information.find_element(By.TAG_NAME, "span").text
        codimap['codimap_title'] = style_list_information.find_element(By.TAG_NAME, "a").get_attribute("title")


        #clas name이 post-information인 div에서 조회수, 댓글수, 날짜 가져오기
        post_information = style_list_item.find_element(By.CLASS_NAME, "post-information")
        spans = post_information.find_elements(By.TAG_NAME, "span")
        codimap['codimap_date'] = spans[0].text
        ##조회수랑 댓글 숫자만 담도록
        views_text = spans[1].text
        views_numeric = int(re.sub("[^0-9]","",views_text)) if views_text else 0 #views_text에서 숫자가 아닌 것은 ""빈카으로 대체
        comments_text = spans[2].text
        comments_numeric = int(re.sub("[^0-9]", "", comments_text)) if comments_text else 0

        codimap['views'] = views_numeric
        codimap['comment_numbers'] = comments_numeric

In [ ]:
df_codimaps = pd.DataFrame(codimaps)
csv_file_path = "codimaps_data.csv"
df_codimaps.to_csv(csv_file_path, index=True, encoding = 'utf-8')

print(f"성공적으로 {csv_file_path}가 저장되었습니다.")

In [38]:
# 심화 도전 과제 버전 
## 크롤링 도중에 에러가 납니다ㅜㅜㅜ
for page in range(1,6):
    url = f"https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page={page}"
    
    driver.get(url)
    driver.maximize_window() # 브라우저의 크기 전체화면으로 확대    
    

    ## 60개의 코디맵 정보를 수집하기 위한 반복문
    codimaps = []
    style_list_items = driver.find_elements(By.CLASS_NAME, "style-list-item")
    
    for style_list_item in style_list_items:
        codimap = {}

        #class name이 style_list_information 인 div에서 codimap_title과 codimap_category 정보 가져오기
        style_list_information = style_list_item.find_element(By.CLASS_NAME, "style-list-information")
        codimap['codimap_category'] = style_list_information.find_element(By.TAG_NAME, "span").text
        codimap['codimap_title'] = style_list_information.find_element(By.TAG_NAME, "a").get_attribute("title")


        #clas name이 post-information인 div에서 조회수, 댓글수, 날짜 가져오기
        post_information = style_list_item.find_element(By.CLASS_NAME, "post-information")
        spans = post_information.find_elements(By.TAG_NAME, "span")
        codimap['codimap_date'] = spans[0].text
        ##조회수랑 댓글 숫자만 담도록
        views_text = spans[1].text
        views_numeric = int(re.sub("[^0-9]","",views_text)) if views_text else 0 #views_text에서 숫자가 아닌 것은 ""빈칸으로 대체
        comments_text = spans[2].text
        comments_numeric = int(re.sub("[^0-9]", "", comments_text)) if comments_text else 0

        codimap['views'] = views_numeric
        codimap['comment_numbers'] = comments_numeric
        
        ##심화 과제 추가 정보 가져오기
        try : 
            style_list_item.find_element(By.CLASS_NAME, "style-list-item__link").click()
            
            #explain, hashtag, imgurl 가져오기
            codimap['codimap_explain'] = driver.find_element(By.CLASS_NAME, "styling_txt").text

            hashtags = driver.find_elements(By.CLASS_NAME, 'ui-tag-list__item')
            codimap['codimap_hashtag'] = [hashtag.text for hashtag in hashtags]

            codimap['codimap_imgurl'] = driver.find_element(By.CLASS_NAME, 'photo').get_attribute('src')
        except NoSuchElementException :
            print("failed")
            
        driver.back()
        codimaps.append(codimap)
    
print(codimaps)

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.200)
Stacktrace:
	GetHandleVerifier [0x00007FF767432142+3514994]
	(No symbol) [0x00007FF767050CE2]
	(No symbol) [0x00007FF766EF76AA]
	(No symbol) [0x00007FF766F079BF]
	(No symbol) [0x00007FF766EFDB89]
	(No symbol) [0x00007FF766EFBF22]
	(No symbol) [0x00007FF766EFF38F]
	(No symbol) [0x00007FF766EFF480]
	(No symbol) [0x00007FF766F41624]
	(No symbol) [0x00007FF766F4197C]
	(No symbol) [0x00007FF766F364FC]
	(No symbol) [0x00007FF766F6602F]
	(No symbol) [0x00007FF766F363B6]
	(No symbol) [0x00007FF766F66490]
	(No symbol) [0x00007FF766F828F6]
	(No symbol) [0x00007FF766F65D93]
	(No symbol) [0x00007FF766F34BDC]
	(No symbol) [0x00007FF766F35C64]
	GetHandleVerifier [0x00007FF76745E16B+3695259]
	GetHandleVerifier [0x00007FF7674B6737+4057191]
	GetHandleVerifier [0x00007FF7674AE4E3+4023827]
	GetHandleVerifier [0x00007FF7671804F9+689705]
	(No symbol) [0x00007FF76705C048]
	(No sy